[[이유한님] 캐글 코리아 캐글 스터디 커널 커리큘럼](https://kaggle-kr.tistory.com/32)

[Statoil/C-CORE Iceberg Classifier Challenge](https://www.kaggle.com/c/statoil-iceberg-classifier-challenge)

[Transfer Learning with VGG-16 CNN+AUG LB 0.1712](https://www.kaggle.com/code/devm2024/transfer-learning-with-vgg-16-cnn-aug-lb-0-1712)

In [1]:
# This Python 3 enviroment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in

import numpy as np  # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directiory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
# print(check_output(["ls", "../input"]).decode("utf8"))    # 윈도우 사용으로 주석처리

# Any results you write to the current directory are saved as output

TL;DR
**Runs on GPU** There is some compatiblity issue with CPUs

1. Hyperparameters in Deep learning are many, tuning them will take weeks or months. Generally researchers do this tuning and publish paper when they find a nice set of architecture which performs better than other.

2. Since the model is pre-trained, it converges very fast and you but still you need GPU to use this. Due to some library issues, it doesn't work on CPU.

3. For our purpose, we can use those architectures, which are made available by those researchers to us.

4. Using those pretrained nets, layers of which already 'knows' how to extract features, we can don't have to tune the hyperparameters. Since they are already trained of some dataset(say imagenet), their pre-trained weights provide a good initialization of weights and because of this, our Convnet converges very fast which otherwise can take days on these deep architectures. That's the idea behind **Transfer Learning**. Examples of which are VGG16, InceptionNet, goolenet, Resnet etc.

5. In this kernel we will use pretrained VGG-16 network which performs very well on small size images.

**VGG architecture has proved to worked well on small sized images(CIFAR-10)** I expected it to work well for this dataset as well.

1. The code also includes the data augmentation steps, thus considerably improving the performance.

2. **GPU is needed**

Here is the link of the research paper if you are interested. https://arxiv.org/pdf/1409.1556.pdf

Also here is the doc for keras library: https://keras.io/applications/#vgg16

**DeepL 번역**  
TL;DR
**GPU에서 실행** CPU와 일부 호환성 문제가 있습니다.

1. 딥 러닝에는 하이퍼파라미터가 많기 때문에 이를 튜닝하는 데 몇 주 또는 몇 달이 걸립니다. 일반적으로 연구자들은 이러한 튜닝을 통해 다른 아키텍처보다 더 나은 성능을 발휘하는 멋진 아키텍처를 발견하면 논문을 발표합니다.

2. 모델이 사전 학습되어 있기 때문에 매우 빠르게 수렴하지만 이를 사용하려면 여전히 GPU가 필요합니다. 일부 라이브러리 문제로 인해 CPU에서는 작동하지 않습니다.

3. 연구자들이 우리에게 제공한 아키텍처를 사용할 수 있습니다.

4. 이미 특징을 추출하는 방법을 '알고' 있는 사전 훈련된 네트워크를 사용하면 하이퍼파라미터를 조정할 필요가 없습니다. 이미 일부 데이터 세트(예: 이미지넷)에 대해 학습되어 있기 때문에 사전 학습된 가중치는 좋은 가중치 초기화를 제공하며, 이 때문에 이러한 심층 아키텍처에서는 며칠이 걸릴 수 있는 Convnet이 매우 빠르게 수렴합니다. 이것이 바로 **Transfer Learning**의 아이디어입니다. 그 예로 VGG16, InceptionNet, goolenet, Resnet 등이 있습니다.

5. 이 커널에서는 작은 크기의 이미지에서 매우 잘 작동하는 사전 학습된 VGG-16 네트워크를 사용할 것입니다.

**VGG 아키텍처는 작은 크기의 이미지(CIFAR-10)에서 잘 작동하는 것으로 입증되었습니다** 이 데이터 세트에서도 잘 작동할 것으로 예상했습니다.

1. 이 코드에는 데이터 증강 단계도 포함되어 있어 성능이 상당히 향상되었습니다.

2. **GPU가 필요합니다**.

관심이 있으시다면 연구 논문 링크를 참고하세요. https://arxiv.org/pdf/1409.1556.pdf

케라스 라이브러리 관련 문서도 여기 있습니다: https://keras.io/applications/#vgg16

In [2]:
# Mandatory imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from os.path import join as opj
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab
# plt.rcParams['figure.figsize'] = 10, 10
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [3]:
train = pd.read_json("./input/005_statoil-iceberg-classifier-challenge/train.json")
target_train = train['is_iceberg']
test = pd.read_json("./input/005_statoil-iceberg-classifier-challenge/test.json")

Keras provide the implementation of pretrained VGG, it in it's library so we don't have to build the net by ourselves. Here we are removing the last layer of VGG and putting our sigmoid layer for binary predictions.

The following code will NOT WORK, since on kaggle notebook, the weights of model cannot be downloaded, however, you can copy paste the code in your own notebook to make it work.

**DeepL 번역**  

Keras는 라이브러리에서 사전 학습된 VGG의 구현을 제공하므로 직접 네트워크를 구축할 필요가 없습니다. 여기서는 VGG의 마지막 레이어를 제거하고 이진 예측을 위한 시그모이드 레이어를 넣습니다.

다음 코드는 kaggle 노트북에서는 모델의 가중치를 다운로드할 수 없기 때문에 작동하지 않지만, 자신의 노트북에 코드를 복사하여 붙여넣으면 작동할 수 있습니다.

In [4]:
target_train = train['is_iceberg']
test['inc_angle'] = pd.to_numeric(test['inc_angle'], errors='coerce')
train['inc_angle'] = pd.to_numeric(train['inc_angle'], errors='coerce') # We have only 133 NAs.
train['inc_angle'] = train['inc_angle'].fillna(method='pad')
X_angle = train['inc_angle']
test['inc_angle'] = pd.to_numeric(test['inc_angle'], errors='coerce')
X_test_angle = test['inc_angle']

# Generate the training data
X_band_1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train['band_1']])
X_band_2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train['band_2']])
X_band_3 = (X_band_1 + X_band_2) / 2
# X_band_3 = np.array([np.full((75, 75), angle).astype(np.float32) for angle in train['inc_angle']])
X_train = np.concatenate([X_band_1[:, :, :, np.newaxis]
                          , X_band_2[:, :, :, np.newaxis]
                          , X_band_3[:, :, :, np.newaxis]], axis=-1)

X_band_test_1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test['band_1']])
X_band_test_2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test['band_2']])
X_band_test_3 = (X_band_test_1 + X_band_test_2) / 2
# X_band_test_3 = np.array([np.full((75, 75), angle).astype(np.float32) for angle in test['inc_angle']])
X_test = np.concatenate([X_band_test_1[:, :, :, np.newaxis]
                          , X_band_test_2[:, :, :, np.newaxis]
                          , X_band_test_3[:, :, :, np.newaxis]], axis=-1)

# Import Keras.
from matplotlib import pyplot
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import GlobalMaxPooling2D
# from keras.layers.normalization import BatchNormalization
from keras.layers.normalization import batch_normalization  # 클래스가 변경된 듯
# from keras.layers.merge import Concatenate
from keras.layers.merging import Concatenate    # 클래스 변경된 듯
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
# from keras.optimizers import rmsprop  # 클래스 제거된 듯, 데스크탑에서 사용
from keras.optimizers import rmsprop_v2   # 랩탑은 환경이 달라 이걸 사용
# from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.layers.activation import LeakyReLU, PReLU    # 호출 경로 변경된 듯
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

from keras.datasets import cifar10
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception
from keras.applications.mobilenet import MobileNet
from keras.applications.vgg19 import VGG19
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

# Data Aug for multi-input
from keras.preprocessing.image import ImageDataGenerator
batch_size = 64
# Define the image transformations here
gen = ImageDataGenerator(horizontal_flip=True,
                         vertical_flip=True,
                         width_shift_range=0.,
                         height_shift_range=0.,
                         channel_shift_range=0,
                         zoom_range=0.2,
                         rotation_range=10)

# Here is the function that merges our two generators
# We use the exact same generator with the same random seed for both the y and angle arrays
def gen_flow_for_two_inputs(X1, X2, y):
    genX1 = gen.flow(X1,y, batch_size=batch_size, seed=55)
    genX2 = gen.flow(X1,X2, batch_size=batch_size, seed=55)
    while True:
            X1i = genX1.next()
            X2i = genX2.next()
            # Assert arrays are equal - this was for peace of mind, but slows down training
            # np.testing.assert_array_equal(X1i[0], X2i[0])
            yield [X1i[0], X2i[1]], X1i[1]

# Finally create generator
def get_callvacks(filepath, patience=2):
      es = EarlyStopping('val_loss', patience=10, mode="min")
      msave = ModelCheckpoint(filepath, save_best_only=True)
      return [es, msave]

def getVggAngleModel():
      input_2 = Input(shape=[1], name="angle")
      angle_layer = Dense(1, )(input_2)
      base_model = VGG16(weights='imagenet', include_top=False,
                         input_shape=X_train.shape[1:])
      x = base_model.get_layer('block5_pool').output

      x = GlobalMaxPooling2D()(x)
      merge_one = concatenate([x, angle_layer])
      merge_one = Dense(512, activation='relu', name='fc2')(merge_one)
      merge_one = Dropout(0.3)(merge_one)
      merge_one = Dense(512, activation='relu', name='fc3')(merge_one)
      merge_one = Dropout(0.3)(merge_one)

      predictions = Dense(1, activation='sigmoid')(merge_one)

      # model = Model(input=[base_model.input, input_2], output=predictions)
      model = Model(inputs=[base_model.input, input_2], outputs=predictions)  # 오기 수정

      # sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
      sgd = SGD(learning_rate=1e-3, decay=1e-6, momentum=0.9, nesterov=True)  # 파라미터 변경
      model.compile(loss='binary_crossentropy',
                    optimizer=sgd,
                    metrics=['accuracy'])
      
      return model

# Using K-fold Cross Validation with Data Augmentation.
def myAngleCV(X_train, X_angle, X_test):
    K=3
    folds = list(StratifiedKFold(n_splits=K, shuffle=True, random_state=16).split(X_train, target_train))
    y_test_pred_log = 0
    y_train_pred_log = 0
    y_valid_pred_log = 0.0 * target_train
    for j, (train_idx, test_idx) in enumerate(folds):
      print('\n===================FOLD=', j)
      X_train_cv = X_train[train_idx]
      y_train_cv = target_train[train_idx]
      X_holdout = X_train[test_idx]
      Y_holdout = target_train[test_idx]

      # Angle
      X_angle_cv = X_angle[train_idx]
      X_angle_hold = X_angle[test_idx]

      # define file path and get callbacks
      file_path = "%s_aug_model_weights.hdf5"%j
      callbacks = get_callvacks(filepath=file_path, patience=5)
      gen_flow = gen_flow_for_two_inputs(X_train_cv, X_angle_cv, y_train_cv)
      galaxyModel = getVggAngleModel()
      galaxyModel.fit_generator(
            gen_flow,
            steps_per_epoch=24,
            epochs=100,
            shuffle=True,
            verbose=1,
            validation_data=([X_holdout, X_angle_hold], Y_holdout),
            callbacks=callbacks)

      # Getting the Best Model
      galaxyModel.load_weights(filepath=file_path)
      # Getting Training Score
      score = galaxyModel.evaluate([X_train_cv, X_angle_cv], y_train_cv, verbose=0)
      print('Train loss:', score[0])
      print('Train accuracy:', score[1])
      # Getting Test Sccore
      score = galaxyModel.evaluate([X_holdout, X_angle_hold], Y_holdout, verbose=0)
      print('Test loss:', score[0])
      print('Test accuracy:', score[1])

      # Getting validation Score:
      pred_valid = galaxyModel.predict([X_holdout, X_angle_hold])
      y_valid_pred_log[test_idx] = pred_valid.reshape(pred_valid.shape[0])

      # Getting Test Scores
      temp_test = galaxyModel.predict([X_test, X_test_angle])
      y_test_pred_log += temp_test.reshape(temp_test.shape[0])

      # Getting Train Scores
      temp_train = galaxyModel.predict([X_train, X_angle])
      y_train_pred_log += temp_train.reshape(temp_train.shape[0])
      
    y_test_pred_log = y_test_pred_log/K
    y_train_pred_log = y_train_pred_log/K

    print('\n Train Log Loss Validation= ', log_loss(target_train, y_train_pred_log))
    print('Test Log Loss Validation= ', log_loss(target_train, y_valid_pred_log))
    return y_test_pred_log

In [5]:
preds = myAngleCV(X_train, X_angle, X_test)


===================FOLD= 0
Epoch 1/100
24/24 [==============================] - 22s 535ms/step - loss: 0.7730 - accuracy: 0.5577 - val_loss: 0.5137 - val_accuracy: 0.7607
Epoch 2/100
24/24 [==============================] - 6s 271ms/step - loss: 0.4856 - accuracy: 0.7594 - val_loss: 0.3137 - val_accuracy: 0.8393
Epoch 3/100
24/24 [==============================] - 6s 260ms/step - loss: 0.3638 - accuracy: 0.8264 - val_loss: 0.2690 - val_accuracy: 0.8692
Epoch 4/100
24/24 [==============================] - 6s 265ms/step - loss: 0.2978 - accuracy: 0.8774 - val_loss: 0.2627 - val_accuracy: 0.8804
Epoch 5/100
24/24 [==============================] - 7s 280ms/step - loss: 0.2863 - accuracy: 0.8778 - val_loss: 0.2675 - val_accuracy: 0.8822
Epoch 6/100
24/24 [==============================] - 7s 301ms/step - loss: 0.2689 - accuracy: 0.8919 - val_loss: 0.2484 - val_accuracy: 0.8953
Epoch 7/100
24/24 [==============================] - 7s 310ms/step - loss: 0.2494 - accuracy: 0.8952 - val_loss: 

In [6]:
# Submission for each day.
submission = pd.DataFrame()
submission['id'] = test['id']
submission['is_iceberg'] = preds
# submission.to_csv('sub.csv', index=False)